In [13]:
import os
import sys
import scipy as sp
import numpy as np
import pandas as pd
import math
import random
import sys

import matplotlib.pyplot as plt
import pylab as pl
import networkx as nx
import cvxopt as co
import sklearn.cluster as cl
from scipy import sparse
from scipy import ndimage
from numpy import unravel_index
from PIL import Image
from sklearn.metrics.pairwise import euclidean_distances
from sklearn import linear_model
from sklearn.svm import SVR
from sklearn import preprocessing
from sklearn.metrics import (median_absolute_error, mean_squared_error, r2_score, mean_absolute_error, 
                             adjusted_rand_score)

from tcrfr_indep_model import TCrfRIndepModel
from tcrfr_pair_model import TCrfRPairwisePotentialModel
from volume import Vol

from regression_methods import (method_ridge_regression, method_transductive_regression, method_svr, method_krr, 
                                method_tkrr, method_lrr, method_tlrr, method_flexmix, method_tcrfr_v2,
                                method_tcrfr_indep, method_tcrfr, evaluate)

In [14]:
%matplotlib inline

In [15]:
# Define a color table
colorTable = ((0,0,143,255), (0,0,159,255), (0,0,175,255), (0,0,191,255), (0,0,207,255),
              (0,0,223,255), (0,0,239,255), (0,0,255,255), (0,16,255,255), (0,32,255,255),
              (0,48,255,255), (0,64,255,255), (0,80,255,255), (0,96,255,255), (0,112,255,255),
              (0,128,255,255), (0,143,255,255), (0,159,255,255), (0,175,255,255), (0,191,255,255),
              (0,207,255,255), (0,223,255,255), (0,239,255,255), (0,255,255,255), (16,255,255,255),
              (32,255,239,255), (48,255,223,255), (64,255,207,255), (80,255,191,255), (96,255,175,255),
              (112,255,159,255), (128,255,143,255), (143,255,128,255), (159,255,112,255), (175,255,96,255),
              (191,255,80,255), (207,255,64,255), (223,255,48,255), (239,255,32,255), (255,255,16,255),
              (255,255,0,255), (255,239,0,255), (255,223,0,255), (255,207,0,255), (255,191,0,255),
              (255,175,0,255), (255,159,0,255), (255,143,0,255), (255,128,0,255), (255,112,0,255),
              (255,96,0,255), (255,80,0,255), (255,64,0,255), (255,48,0,255), (255,32,0,255),
              (255,16,0,255), (255,0,0,255), (239,0,0,255), (223,0,0,255), (207,0,0,255),
              (191,0,0,255), (175,0,0,255), (159,0,0,255), (143,0,0,255))
    
# Plot an image
def plot_image(mat, colorTable=colorTable):
    
    # Get image dimensions
    (x,y) = mat.shape
    
    # Get the maximum and minimum values
    minValue = np.amin(mat)
    maxValue = np.amax(mat)

    # Define min and max color indexes
    minIndex = 0
    maxIndex = len(colorTable)-1
    
    # Generate the color index volume
    img_color = np.zeros((x,y), dtype=int)
    img_color = np.round((mat-minValue)/(maxValue-minValue)*(maxIndex-minIndex)+minIndex).astype(int)
        
    # Create and display image
    img = Image.new( 'RGBA', (x,y), "white") # create a new white image
    pixels = img.load() # create the pixel map

    for i in range(img.size[0]):  # for every pixel
        for j in range(img.size[1]):
            pixels[i,j] = (colorTable[img_color[i,j]][0], 
                           colorTable[img_color[i,j]][1], 
                           colorTable[img_color[i,j]][2], 
                           colorTable[img_color[i,j]][3])
    plt.imshow(np.array(img), origin='lower')

In [16]:
# Load ip, porosity, and facies images
ip = Vol()
phi = Vol()
ip.read("../Porest-Data/Real_data/acoustic_impedance_real_data.vol")
phi.read("../Porest-Data/Real_data/porosity_real_data.vol")

In [17]:
'''
# Add some noise to the input data
mu = 0.
sigma = 100.0
noise = np.random.normal(mu, sigma, ip.data.shape)
ip.data +=(noise)
#noise = np.random.normal(mu, sigma, si.data.shape)
si.data +=(noise)
#noise = np.random.normal(mu, sigma, pr.data.shape)
pr.data +=(noise)
'''

'\n# Add some noise to the input data\nmu = 0.\nsigma = 100.0\nnoise = np.random.normal(mu, sigma, ip.data.shape)\nip.data +=(noise)\n#noise = np.random.normal(mu, sigma, si.data.shape)\nsi.data +=(noise)\n#noise = np.random.normal(mu, sigma, pr.data.shape)\npr.data +=(noise)\n'

In [18]:
# Run the experiment
sizeX = phi.sizeX
sizeY = phi.sizeY
#sizeZ = phi.sizeZ
sizeZ = 1
null_value = -1e+30
train_frac = 0.02
slice_number = 35
n_train_wells = int(sizeX*sizeY*train_frac) # number of training wells
RL = 10 # max radius distance from labeled node to neighbors
RL_x = RL; RL_y = RL; RL_z = 1 # radius distance from labeled node in x,y and z directions
RU = 1 # max radius distance from unlabeled node to neighbors
RU_x = RU; RU_y = RU; RU_z = 1 # radius distance from unlabeled node in x,y and z directions
rel_fac = 0.5 # similarity factor to check if two neighbor samples should be connected by an edge
sim_fac = 0.95 # similarity factor to check if two neighbor samples should have the same latent state
states = 3 # number of facies
REPS = 5 # number of repetitions

# Create the ground truth images
z = slice_number
ip_mat = np.zeros((sizeX,sizeY))
phi_mat = np.zeros((sizeX,sizeY))
facies_mat = np.zeros((sizeX,sizeY),dtype=int)
for x in range(sizeX):
    for y in range(sizeY):
        ip_mat[x,y] = ip.data[z,y,x]
        phi_mat[x,y] = phi.data[z,y,x]
        
# Create arrays to store estimated errors and images
errs = np.zeros((REPS,6,6))
flx_facies_mat = np.zeros((REPS, sizeX,sizeY))
flx_phi_mat = np.zeros((REPS, sizeX,sizeY))
svr_phi_mat = np.zeros((REPS, sizeX,sizeY))
rr_phi_mat = np.zeros((REPS, sizeX,sizeY))
tr_phi_mat = np.zeros((REPS, sizeX,sizeY))
tcrfr_facies_mat = np.zeros((REPS, sizeX,sizeY))
tcrfr_phi_mat = np.zeros((REPS, sizeX,sizeY))
shale_coef = np.zeros(REPS)

# Build the graph for the adjacency matrix
print 'creating the unlabeled grid...',
sys.stdout.flush()
G = nx.Graph()
node_list = []
cz = slice_number
var = np.var(ip.data)
for cx in range(sizeX):
    for cy in range(sizeY):
        G.add_node((cx,cy,cz))
        node_list.append((cx,cy,cz))
        u = [cx,cy,cz]
        for i in range(cx,cx+RU_x+1):
            for j in range(cy-RU_y,cy+RU_y+1):
                k = cz
                if (i >= 0 and i < sizeX) and (j >= 0 and j < sizeY):
                    v = [i,j,k]
                    dist = 1.
                    rel = math.exp(-(si.data[cz,cy,cx]-si.data[k,j,i])**2/(2*var))/dist
                    if rel >= rel_fac:
                        d = euclidean_distances(u,v)
                        if d > 0. and d <= RU:
                            G.add_edge((cx,cy,cz),(i,j,k),weight=2)
print 'done!'
sys.stdout.flush()

creating the unlabeled grid...

AttributeError: Vol instance has no attribute 'data'

In [ ]:
e_bunch = []
for r in range(REPS):
    print 'Iteration '+ str(r)
    print 'creating the train and test sets...',
    sys.stdout.flush()    
    # Set input features and output
    X = []
    y = []
    facies = []
    vol_inds = []
    cz = slice_number
    n_samples = 0
    for i in range(sizeX): 
        for j in range(sizeY):
            if ip.data[k,j,i] > null_value:
                X.append([ip.data[cz,j,i]])
                y.append(phi.data[cz,j,i])
                vol_inds.append((i,j,cz))
                n_samples += 1
       
    # Obtain the labeled data (from the "wells")
    test = list(np.arange(n_samples))
    img_inds = list(np.ndindex((sizeX,sizeY)))
    inds = list(test)
    np.random.shuffle(inds)
    img_size = sizeX*sizeY

    train = np.array(inds[:n_train_wells])
    test = np.array(inds[n_train_wells:])
    X = np.array(X, copy=True)
    y = np.concatenate(np.array(y, copy=True).reshape((-1,1)))
    print 'done!'
    sys.stdout.flush()

    # Add edges for the labeled data
    print 'adding new edges for the labeled data...',
    sys.stdout.flush()
    G.remove_edges_from(e_bunch)
    e_bunch = []
    for l in range(len(train)):
        cx = vol_inds[train[l]][0]
        cy = vol_inds[train[l]][1]
        u = [cx,cy,cz]
        for i in range(cx,cx+RL_x+1):
            for j in range(cy-RL_y,cy+RL_y+1):
                k = cz
                if (i >= 0 and i < sizeX) and (j >= 0 and j < sizeY):
                    v = [i,j,k]
                    dist = 1.
                    rel = math.exp(-(si.data[cz,cy,cx]-si.data[k,j,i])**2/(2*var))/dist
                    if rel >= rel_fac:
                        d = euclidean_distances(u,v)
                        if d > 0. and d <= RL:
                            if not G.has_edge((cx,cy,cz),(i,j,k)):
                                G.add_edge((cx,cy,cz),(i,j,k),weight=2)
                                e_bunch.append(((cx,cy,cz),(i,j,k)))
    print 'done!'
    sys.stdout.flush()
    
    # Create the sparse matrix
    print 'building the sparse matrix...',
    sys.stdout.flush()   
    csr_A = nx.adjacency_matrix(G,nodelist=node_list)
    coo = csr_A.tocoo()
    A = co.spmatrix(1, coo.row.tolist(), coo.col.tolist())
    print 'done!'
    sys.stdout.flush()
        
    # Define some auxiliary arrays
    n_test = len(test)
    pred_flx = np.zeros(n_test)
    pred_svr = np.zeros(n_test)
    pred_rr = np.zeros(n_test)
    pred_tr = np.zeros(n_test)
    pred_tcrfr = np.zeros(n_test)
    
    # Normalize the data with zero mean and unit variance
    X = X - np.mean(X[train,:])
    y = y - np.mean(y[train])
    X /= np.max(np.abs(X[train,:]))
    y /= np.max(np.abs(y[train]))
    y *= 10.  
    
    # Add the intercept to X
    X = np.hstack((X, np.ones((X.shape[0],1))))
    feats = X.shape[1] # number of features

    # Save stacked data
#    xx = np.hstack((X,y.reshape(-1,1)))
#    np.savetxt("../GeoFacies-Data/vol_real_data.txt",xx)
               
    # Run the methods
    print 'running flexmix...',
    sys.stdout.flush()
    (name, pred_flx, lats_flx, lats_flx_all) = method_flexmix(np.array(X, copy=True), np.array(y, copy=True), 
                                                                       train, test, states=states, 
                                                                       params=[700, 0.01])
    print 'done!'
    sys.stdout.flush()
    
    print 'running svr...',
    sys.stdout.flush()   
    (name, pred_svr, lats_svr) = method_svr(np.array(X, copy=True), np.array(y, copy=True), 
                                                     train, test, states=states, 
                                                     params=[100.,1.,'linear'])
    print 'done!'
    sys.stdout.flush()
    
    print 'running rr...',
    sys.stdout.flush()
    (name, pred_rr, lats_rr) = method_ridge_regression(np.array(X, copy=True), np.array(y, copy=True), 
                                                                train, test, states=states, 
                                                                params=0.01)
    print 'done!'
    sys.stdout.flush()
    
    print 'running tr...',
    sys.stdout.flush()
    (name, pred_tr, lats_tr) = method_transductive_regression(np.array(X, copy=True), np.array(y, copy=True), 
                                                                       train, test, states=states, 
                                                                       params=[0.000001, 1000., 0.1])
    print 'done!'
    sys.stdout.flush()
    
    print 'running tcrfr...',
    sys.stdout.flush()
    (name, pred_tcrfr, lats_tcrfr, lats_tcrfr_all) = method_tcrfr_v2(np.array(X, copy=True), np.array(y, copy=True), 
                                                                      train, test, A, states=states, 
                                                                      params=[0.8, 0.00001, 1000.0], plot=False)    
    print 'done!'
    sys.stdout.flush()
    
    # Evaluate the performance
    foo = [0,0]
    (errs[r,0, :],_) = evaluate(y[test], pred_flx, foo, foo)
    (errs[r,1, :],_) = evaluate(y[test], pred_svr, foo, foo)
    (errs[r,2, :],_) = evaluate(y[test], pred_rr, foo, foo)
    (errs[r,3, :],_) = evaluate(y[test], pred_tr, foo, foo)
    (errs[r,4, :],_) = evaluate(y[test], pred_tcrfr, foo, foo) 

    # Store image results
    for rk in range(len(train)):
        ind = train[rk]
        ri = vol_inds[ind][0]
        rj = vol_inds[ind][1]
#        flx_facies_mat[r,ri,rj] = lats_flx_all[ind]
        flx_phi_mat[r,ri,rj] = y[ind]
        svr_phi_mat[r,ri,rj] = y[ind]
        rr_phi_mat[r,ri,rj] = y[ind]
        tr_phi_mat[r,ri,rj] = y[ind]
        tcrfr_facies_mat[r,ri,rj] = lats_tcrfr_all[ind]
        tcrfr_phi_mat[r,ri,rj] = y[ind]
    for rk in range(len(test)):
        ind = test[rk]
        ri = vol_inds[ind][0]
        rj = vol_inds[ind][1]        
#        flx_facies_mat[r,ri,rj] = lats_flx_all[ind]
        flx_phi_mat[r,ri,rj] = pred_flx[rk]
        svr_phi_mat[r,ri,rj] = pred_svr[rk]
        rr_phi_mat[r,ri,rj] = pred_rr[rk]
        tr_phi_mat[r,ri,rj] = pred_tr[rk]
        tcrfr_facies_mat[r,ri,rj] = lats_tcrfr_all[ind]
        tcrfr_phi_mat[r,ri,rj] = pred_tcrfr[rk]


In [ ]:
print 'number of training wells: ', n_train_wells

In [ ]:
# Get the averages
err_avg = np.mean(errs,axis=0)
flx_phi_mat_avg = np.mean(flx_phi_mat,axis=0)
svr_phi_mat_avg = np.mean(svr_phi_mat,axis=0)
rr_phi_mat_avg = np.mean(rr_phi_mat,axis=0)
tr_phi_mat_avg = np.mean(tr_phi_mat,axis=0)
tcrfr_phi_mat_avg = np.mean(tcrfr_phi_mat,axis=0)

In [ ]:
print ("          MAE        MSE        RMSE       MDAE        R2        ARS")
print ("FLX:    %0.5f    %0.5f    %0.5f    %0.5f    %0.5f    %0.5f" % (err_avg[0,0],err_avg[0,1],err_avg[0,2],err_avg[0,3],err_avg[0,4],err_avg[0,5]))
print ("SVR:    %0.5f    %0.5f    %0.5f    %0.5f    %0.5f    %0.5f" % (err_avg[1,0],err_avg[1,1],err_avg[1,2],err_avg[1,3],err_avg[1,4],err_avg[1,5]))
print ("RR:     %0.5f    %0.5f    %0.5f    %0.5f    %0.5f    %0.5f" % (err_avg[2,0],err_avg[2,1],err_avg[2,2],err_avg[2,3],err_avg[2,4],err_avg[2,5]))
print ("TR:     %0.5f    %0.5f    %0.5f    %0.5f    %0.5f    %0.5f" % (err_avg[3,0],err_avg[3,1],err_avg[3,2],err_avg[3,3],err_avg[3,4],err_avg[3,5]))
print ("TCRFR:  %0.5f    %0.5f    %0.5f    %0.5f    %0.5f    %0.5f" % (err_avg[4,0],err_avg[4,1],err_avg[4,2],err_avg[4,3],err_avg[4,4],err_avg[4,5]))

In [ ]:
# Plot the facies image results
r = 0
f = pl.figure(figsize=(18, 14))
ax = f.add_subplot(1, 3, 1)
plot_image(ip_mat)
ax.set_title('Input data')
ax = f.add_subplot(1, 3, 2)
color_table = ((0,0,143,255), (255,200,0,255))
plot_image(facies_mat, colorTable=color_table)
ax.set_title('Ground truth')
ax = f.add_subplot(1, 3, 3)
plot_image(tcrfr_facies_mat[r], colorTable=color_table)
ax.set_title('TCRFR')
pl.show()

In [ ]:
# Plot the porosity image results
f = pl.figure(figsize=(18, 14))
ax = f.add_subplot(1, 5, 1)
plot_image(phi_mat)
ax.set_title('Ground truth')
ax = f.add_subplot(1, 5, 2)
plot_image(flx_phi_mat_avg)
ax.set_title('FLX')
ax = f.add_subplot(1, 5, 3)
plot_image(tr_phi_mat_avg)
ax.set_title('SVR')
ax = f.add_subplot(1, 5, 4)
plot_image(tr_phi_mat_avg)
ax.set_title('TR')
ax = f.add_subplot(1, 5, 5)
plot_image(tcrfr_phi_mat_avg)
ax.set_title('TCRFR')
pl.show()

In [ ]:
# Plot the fitting curves for the last iteration
sinds = np.argsort(X[test,0])
f = pl.figure(figsize=(21, 7))
ax = f.add_subplot(1, 3, 1)
plt.plot(X[test[sinds],0], y[test[sinds]], 'or', alpha=1.0, markersize=10.0)
plt.plot(X[test[sinds],0], pred_flx[sinds], 'ob', alpha=0.9, markersize=6.0)
plt.legend(['Truth', 'FLX'], loc=1, fontsize=10)
ax.set_title('FlexMix Fitting')
ax = f.add_subplot(1, 3, 2)
plt.plot(X[test[sinds],0], y[test[sinds]], 'or', alpha=1.0, markersize=10.0)
plt.plot(X[test[sinds],0], pred_tr[sinds], 'ob', alpha=0.9, markersize=6.0)
plt.legend(['Truth', 'TR'], loc=1, fontsize=10)
ax.set_title('Transductive Regression Fitting')
ax = f.add_subplot(1, 3, 3)
plt.plot(X[test[sinds],0], y[test[sinds]], 'or', alpha=1.0, markersize=10.0)
plt.plot(X[test[sinds],0], pred_tcrfr[sinds], 'ob', alpha=0.9, markersize=6.0)
plt.legend(['Truth', 'TCRFR'], loc=1, fontsize=10)
ax.set_title('Transductive CRF Fitting')
pl.show()